<a href="https://colab.research.google.com/github/JoelYanotka/text-summary-to-speech/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-j64k66ya
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-j64k66ya
  Resolved https://github.com/huggingface/transformers to commit f0f5e28f8221e2733208ec49badcb1ba03c6a49b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9

In [2]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=f8d7b324d8f0cac205cc9f4d804cbd79c5fd0c735c88fe5bfca340e1692560fe
  Stored in directory: /root/.cache/pip/wheels/c2/46/f4/caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


In [3]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [4]:
import re

import wikipedia
from urllib.request import urlopen
from bs4 import BeautifulSoup

def summarize_page(wiki_title):
    '''
    Retrieves a Wikipedia page with the title given in the input, and extracts
    its sections and paragraphs. Then, it generates summaries of the paragraphs.
    Parameters:
        wiki_title: a string with the title of a Wikipedia page.
    Output:
        A list of strings with summaries of the sections of the Wikipedia page.
    '''
    # Get a Wikipedia page by its title
    wiki = wikipedia.page(wiki_title)

    # Get a Wikipedia page by its url and make a soup
    source = urlopen(wiki.url).read()
    soup = BeautifulSoup(source,'lxml')

    # page will contain the list of sections in the page, as delimited by html headlines
    # Add a list with the page title as the first element and the summary as the second
    page = [[soup.find('h1').get_text(), wiki.summary]]

    for header in soup.find_all(['h2', 'h3']):
        header_name = header.get_text().replace('[edit]', '')
        if header.get_text() == 'Contents':
            continue
        if 'References' in header.get_text():
            break
        # Every element in the section list will be a list with the name of the
        # headline as the first element and the paragraphs as the next elements
        section = [header_name]
        for elem in header.next_siblings:
            # Stop at next header
            if elem.name and elem.name.startswith('h'):
                break
            if elem.name == 'p':
                # re.sub eliminate references
                paragraph = re.sub(r'\[.*?\]+', '', elem.get_text())\
                    .replace('\n', '')\
                    .replace(u'\xa0', ' ')
                section.append(paragraph)
        page.append(section)

    summaries = []
    for i, section in enumerate(page):
        if len(section) > 1:
            for j, paragraph in enumerate(section[1:]):
                lenght = len(paragraph.split())
                summary = summarizer(paragraph,
                                     max_length=lenght,
                                     min_length=2,
                                     do_sample=False)
                summaries.append(summary[0]['summary_text'])
                print(f'\rSection: {i+1}/{len(page)}\tParagraph: {j+1} / {len(section)}', end='')
    print('\r')
    return summaries

Because the model does not convert numerical characters and abbreviations into speech, the following steps are required in order for the model to be able to pronounce numbers and measurements correctly.

In [6]:
!pip install num2words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=9c4127b5c3c06c87abae9bedcf9594672f8cae9c9d18ac0af391aff995c670bc
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [7]:
import re
import num2words

def parse_numbers(text):
    '''
    Takes a string text as input, and returns a modified version of the input
    string where the numbers in the text have been converted to their textual
    form and measurement abbreviations have been replaced for their full name.
    '''
    # Common measurement abbreviations to replace for its full name
    measurements = {
    "mm": "milimeters",
    "cm": "centimeters",
    "m": "meters",
    "km": "kilometers",
    "mg": "milligrams",
    "g": "grams",
    "kg": "kilograms",
    "ml": "milliters",
    "l": "liters",
    "L": "liters",
    "in": "inches",
    "ft": "feet",
    "yd": "yards",
    "mi": "miles",
    "oz": "ounces",
    "lb": "pounds",
    "gal": "gallons"
}
    
    # Find numbers folowed by any of the previous measurement abbreviations and
    # replace the abbreviation for the full name
    pattern = r'(\d+)\s*(' + '|'.join(measurements.keys()) + r')\b'
    text = re.sub(pattern,
                  lambda m: m.group(1) + ' ' + measurements[m.group(2)],
                  text)

    # Eliminate the leading zeroes in a decimal number
    text = re.sub(r"(\.\d*?[1-9])0+\b", r"\1", text)

    # Find the numbers with comma separators and deletes the comma
    matches = re.findall(r"(\d+,\d+)", text)
    for match in matches:
        text = text.replace(match, match.replace(",", ""))

    # Find the decimal numbers and replaces the dot character for the word "point"
    matches = re.findall(r"(\d+.\d+)", text)
    for match in matches:
        text = text.replace(match, match.replace(".", " point "))

    # Replace numbers for words
    text = re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), text)

    return text

Now the text is ready to be converted into audio.
Due to the fact that the text-to-speech model supports up to 600 words, it is neccesary to create many audio files and then combine them to create the final product.

In [8]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

def text2audio(text, output_name='output.wav', processor=processor, model=model, vocoder=vocoder):
    '''
    Converts a given text into an audio file. The function uses a
    text-to-speech model to synthesize speech and a vocoder to convert the
    generated speech into an audio waveform.
    '''
    inputs = processor(text=text, return_tensors="pt")
    # load xvector containing speaker's voice characteristics from a dataset
    embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
    speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

    speech = model.generate_speech(inputs["input_ids"],
                                   speaker_embeddings,
                                   vocoder=vocoder)
    sf.write(output_name, speech.numpy(), samplerate=16000)

In [9]:
import os
import wave

def merge_audio(list_of_text, out_dir=''):
    '''
    Takes a list of text strings as input and outputs a merged audio file
    containing the spoken versions of the text.
    Parameters:
        list_of_text (List[str]): A list of text strings to be converted to
        speech and merged together.
        out_dir (str): The destination directory for the audio file.
        If not specified, it will be created in the current directory.
    Return:
        None. The merged audio file is saved in the specified directory.
    '''
    for i, line in enumerate(list_of_text):
        text2audio(text=line, output_name=f'tmp_output_{i:03d}.wav')

    infiles = [f for f in os.listdir() if f.startswith('tmp_output_')]
    infiles.sort()
    outfile = out_dir + "output.wav"

    data= []
    for infile in infiles:
        w = wave.open(infile, 'rb')
        data.append([w.getparams(), w.readframes(w.getnframes())])
        w.close()
        
    output = wave.open(outfile, 'wb')
    output.setparams(data[0][0])
    for i in range(len(data)):
        output.writeframes(data[i][1])
    output.close()

    for f in infiles:
        os.remove(f)

In [10]:
if __name__ == '__main__':
    wiki_title = input('Type the full title of a Wikipedia page: ')
    wiki_title = wiki_title.strip().replace(' ', '_')
    
    summarized_text = summarize_page(wiki_title)
    summarized_text = [parse_numbers(line) for line in summarized_text]

    merge_audio(summarized_text)

Type the full title of a Wikipedia page:  2023 London Marathon 
Section: 6/11	Paragraph: 2 / 3


Generating validation split: 0 examples [00:00, ? examples/s]

Dataset cmu-arctic-xvectors downloaded and prepared to /root/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f. Subsequent calls will reuse this data.
